#  Entity Relation Classification  with SciBERT

This model is the same as Entity Relation Classification with BERT except that we get the BERT model which is pre-trained in a scientif corpus.






###  Import neccesarry modules

In [1]:
import warnings
warnings.filterwarnings('ignore')

import io
import random
import numpy as np
import mxnet as mx
import gluonnlp as nlp
from Bert import data, model

### Setting up the environment

In [3]:
np.random.seed(100)
random.seed(100)
mx.random.seed(10000)
ctx = mx.cpu(0)

## Using the pre-trained BERT model



 
"The BERT model we will use is BERT
BASE trained in the scientific corpus. 'scibert_scivocab_uncased'"

### Get BERT



Load the pre-trained BERT model. 

In [4]:
bert_base, vocabulary = nlp.model.get_model('bert_12_768_12',
                                             dataset_name='scibert_scivocab_uncased',
                                             pretrained=True, ctx=ctx, use_pooler=True,
                                             use_decoder=False, use_classifier=False)
print(bert_base)

BERTModel(
  (encoder): BERTEncoder(
    (dropout_layer): Dropout(p = 0.1, axes=())
    (layer_norm): BERTLayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
    (transformer_cells): HybridSequential(
      (0): BERTEncoderCell(
        (dropout_layer): Dropout(p = 0.1, axes=())
        (attention_cell): MultiHeadAttentionCell(
          (_base_cell): DotProductAttentionCell(
            (_dropout_layer): Dropout(p = 0.1, axes=())
          )
          (proj_query): Dense(768 -> 768, linear)
          (proj_key): Dense(768 -> 768, linear)
          (proj_value): Dense(768 -> 768, linear)
        )
        (proj): Dense(768 -> 768, linear)
        (ffn): BERTPositionwiseFFN(
          (ffn_1): Dense(768 -> 3072, linear)
          (activation): GELU()
          (ffn_2): Dense(3072 -> 768, linear)
          (dropout_layer): Dropout(p = 0.1, axes=())
          (layer_norm): BERTLayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
        )
        (la

### Add the Classification layer, BERTClassifier

1. The original model was trained for sentence pair classificaton. 
2. Entity + Context pair will be fed to the model. 
3. Both entities with context will be treated as sentences and the model will try to predict their label. 
4. After BERT model is loaded, another layer for classification (the BERTClassifier)is attached to it.  
5. This works as follows: The `BERTClassifier` class encodes the entity representations by using a BERT base model then uses a `nn.Dense` layer for classification.

In [5]:
#Add the classification layer to the bert_base
bert_classifier = model.classification.BERTClassifier(bert_base, num_classes=6, dropout=0.1)

# only need to initialize the classifier layer.
bert_classifier.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
bert_classifier.hybridize(static_alloc=True)

# softmax cross entropy loss for classification
loss_function = mx.gluon.loss.SoftmaxCELoss()
loss_function.hybridize(static_alloc=True)

metric = mx.metric.Accuracy()

## Data preprocessing for BERT


### Loading the dataset

1. Import the pickeled file of entity pair in a dataframe.
2. Transform the dataframe into a tsv(tab separated values) file
3. Read the tsv file and print some examples to have a feeling how does data look like before it is fed into model.

In [6]:
tsv_file = io.open('train.tsv', encoding='utf-8')
for i in range(5):
    print(tsv_file.readline())

entity_text_1	entity_text_2	label

database traditional information retrieval techniques use a	histogram of keywords as the	USAGE

representation but oral communication may offer	offer additional indices such as	USAGE

a large database of tv	database of tv shows emotions and	PART_WHOLE

of a distributed message-passing infrastructure for dialogue	infrastructure for dialogue systems which all	MODEL-FEATURE



In [7]:
# Skip the first line, column names
num_discard_samples = 1
# Split fields by tabs
field_separator = nlp.data.Splitter('\t')
# Fields to select from the file
field_indices = [0, 1, 2]
train_data = nlp.data.TSVDataset(filename='train.tsv',
                                 field_separator=field_separator,
                                 num_discard_samples=num_discard_samples,
                                 field_indices=field_indices)
sample_id = 0
# First entity context
print(train_data[sample_id][0])
# Second enitiy context
print(train_data[sample_id][1])
# relation type (label)
print(train_data[sample_id][2])

database traditional information retrieval techniques use a
histogram of keywords as the
USAGE


We will use
`BERTDatasetTransform` to perform the following transformations:
- tokenize
the
input sequences
- insert [CLS] at the beginning
- insert [SEP] between entities and at the end
- generate segment ids to indicate whether a token belongs to the first sequence or the second sequence.
- generate valid length

In [8]:
# Use the vocabulary from pre-trained model for tokenization
bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=True)

# The maximum length of an input sequence, the averge length of the entity
max_len = 12

all_labels = ["USAGE", "PART_WHOLE", "MODEL-FEATURE", "RESULT", "COMPARE", "TOPIC"]

pair = True
transform = data.transform.BERTDatasetTransform(bert_tokenizer, max_len,
                                                class_labels=all_labels,
                                                has_label=True,
                                                pad=True,
                                                pair=pair)
data_to_train = train_data.transform(transform)

In [22]:
type(data_to_train)

mxnet.gluon.data.dataset._LazyTransformDataset

1. Followings are printout of how the BERT vocabulary looks like, with the vocabulary size and special tokens. 
2. The ids of the special tokens


In [9]:
print('vocabulary used for tokenization = %s'%vocabulary)
print('%s token id = %s'%(vocabulary.padding_token, vocabulary[vocabulary.padding_token]))
print('%s token id = %s'%(vocabulary.cls_token, vocabulary[vocabulary.cls_token]))
print('%s token id = %s'%(vocabulary.sep_token, vocabulary[vocabulary.sep_token]))
print('token ids = %s'%data_to_train[sample_id][0])
print('valid length = %s'%data_to_train[sample_id][1])
print('segment ids = %s'%data_to_train[sample_id][2])
print('label = %s'%data_to_train[sample_id][3])


vocabulary used for tokenization = Vocab(size=31090, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")
[PAD] token id = 0
[CLS] token id = 102
[SEP] token id = 103
token ids = [  102  3139  3783   776  6606  2190   103 11261   131  4302   188   103]
valid length = 12
segment ids = [0 0 0 0 0 0 0 1 1 1 1 1]
label = [0]


## Fine-tuning the model

In [10]:
# The hyperparameters
batch_size = 32
lr = 5e-6

# The FixedBucketSampler and the DataLoader for making the mini-batches
train_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[1]) for item in data_to_train],
                                            batch_size=batch_size,
                                            shuffle=True)
bert_dataloader = mx.gluon.data.DataLoader(data_to_train, batch_sampler=train_sampler)

trainer = mx.gluon.Trainer(bert_classifier.collect_params(), 'adam',
                           {'learning_rate': lr, 'epsilon': 1e-9})

# Collect all parameters
params = [p for p in bert_classifier.collect_params().values() if p.grad_req != 'null']
grad_clip = 1

In [18]:
# Training the model with different epochs 10, 5
# The output is when trained for 5 epochs 
# In total it is trained on 15 epochs
log_interval = 4
num_epochs = 5

for epoch_id in range(num_epochs):
    metric.reset()
    step_loss = 0 # for outputing the loss that happens in each step. 
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(bert_dataloader):
        with mx.autograd.record():

            # Load the data 
            token_ids = token_ids.as_in_context(ctx)
            valid_length = valid_length.as_in_context(ctx)
            segment_ids = segment_ids.as_in_context(ctx)
            label = label.as_in_context(ctx)

            # Forward computation
            out = bert_classifier(token_ids, segment_ids, valid_length.astype('float32'))
            ls = loss_function(out, label).mean()

        # Backwards computation
        ls.backward()

        # Gradient clipping
        trainer.allreduce_grads()
        nlp.utils.clip_grad_global_norm(params, 1)
        trainer.update(1)

        step_loss += ls.asscalar()
        metric.update([label], [out])

        # Printing vital information
        if (batch_id + 1) % (log_interval) == 0:
            print('[Epoch {} Batch {}/{}] loss={:.4f}, lr={:.7f}, acc={:.3f}'
                         .format(epoch_id, batch_id + 1, len(bert_dataloader),
                                 step_loss / log_interval,
                                 trainer.learning_rate, metric.get()[1]))
            step_loss = 0

[Epoch 0 Batch 4/40] loss=0.3299, lr=0.0000050, acc=0.938
[Epoch 0 Batch 8/40] loss=0.2878, lr=0.0000050, acc=0.948
[Epoch 0 Batch 12/40] loss=0.5241, lr=0.0000050, acc=0.919
[Epoch 0 Batch 16/40] loss=0.3981, lr=0.0000050, acc=0.918
[Epoch 0 Batch 20/40] loss=0.2608, lr=0.0000050, acc=0.927
[Epoch 0 Batch 24/40] loss=0.3622, lr=0.0000050, acc=0.926
[Epoch 0 Batch 28/40] loss=0.4300, lr=0.0000050, acc=0.920
[Epoch 0 Batch 32/40] loss=0.2648, lr=0.0000050, acc=0.920
[Epoch 0 Batch 36/40] loss=0.5383, lr=0.0000050, acc=0.913
[Epoch 0 Batch 40/40] loss=0.4580, lr=0.0000050, acc=0.910
[Epoch 1 Batch 4/40] loss=0.3010, lr=0.0000050, acc=0.945
[Epoch 1 Batch 8/40] loss=0.4318, lr=0.0000050, acc=0.898
[Epoch 1 Batch 12/40] loss=0.2966, lr=0.0000050, acc=0.914
[Epoch 1 Batch 16/40] loss=0.2404, lr=0.0000050, acc=0.926
[Epoch 1 Batch 20/40] loss=0.2316, lr=0.0000050, acc=0.931
[Epoch 1 Batch 24/40] loss=0.2964, lr=0.0000050, acc=0.927
[Epoch 1 Batch 28/40] loss=0.2551, lr=0.0000050, acc=0.929
[

## Upload and Transform Testing Data
Do the same pre-processing as we did for training data. 

In [12]:
#read the test file 
test_file = io.open('test_data_with_labels.tsv', encoding='utf-8')
for i in range(5):
    print(test_file.readline())

entity_context_1	entity_context_2	label

labeling (srl) arguments are usually	in a syntax subtree it arguments	PART_WHOLE

to label arguments locally in	such a sub-tree rather than	PART_WHOLE

approach the anchor group approach achieves an	achieves an accuracy of 87.75%	RESULT

that the prediction of mp improves semantic	mp improves semantic role labeling	RESULT



In [13]:
# Skip the first line, column names
num_discard_samples = 1
# Split fields by tabs
field_separator = nlp.data.Splitter('\t')
# Fields to select from the file
field_indices = [0, 1, 2]
test_data = nlp.data.TSVDataset(filename='test_data_with_labels.tsv',
                                 field_separator=field_separator,
                                 num_discard_samples=num_discard_samples,
                                 field_indices=field_indices)

#printout an example of testing data.
sample_id = 3
# First entity context
print(test_data[sample_id][0])
# Second enitiy context
print(test_data[sample_id][1])
#label
print(test_data[sample_id][2])

that the prediction of mp improves semantic
mp improves semantic role labeling
RESULT


In [14]:
# Use the vocabulary from pre-trained model for tokenization
bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=True)

# The maximum length of an input sequence as we have specified in previous examples
max_len = 12
all_labels = ["USAGE", "PART_WHOLE", "MODEL-FEATURE", "RESULT", "COMPARE", "TOPIC"]
pair = True
transform = data.transform.BERTDatasetTransform(bert_tokenizer, max_len,
                                                class_labels=all_labels,
                                                has_label=True,
                                                pad=True,
                                                pair=pair)

test = test_data.transform(transform)

In [15]:
print('vocabulary used for tokenization = %s'%vocabulary)
print('%s token id = %s'%(vocabulary.padding_token, vocabulary[vocabulary.padding_token]))
print('%s token id = %s'%(vocabulary.cls_token, vocabulary[vocabulary.cls_token]))
print('%s token id = %s'%(vocabulary.sep_token, vocabulary[vocabulary.sep_token]))
print('token ids = %s'%test[sample_id][0])
print('valid length = %s'%test[sample_id][1])
print('segment ids = %s'%test[sample_id][2])
print('label = %s'%test[sample_id][3])

vocabulary used for tokenization = Vocab(size=31090, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")
[PAD] token id = 0
[CLS] token id = 102
[SEP] token id = 103
token ids = [ 102  198  111 3838  131 6051  103 6051 8112 5437 1447  103]
valid length = 12
segment ids = [0 0 0 0 0 0 0 1 1 1 1 1]
label = [3]


## Predicting the label

In [20]:
# Set the hyperparameters
batch_size = 1

# Take the rows from the test data one by one  
test_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[1]) for item in test],
                                           batch_size=batch_size, shuffle=False)
# Load the test data
test_dataloader = mx.gluon.data.DataLoader(test, batch_sampler=test_sampler)

In [21]:
probabilities = []
metric.reset()
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):

    # Load the data
    token_ids = token_ids.as_in_context(ctx)
    valid_length = valid_length.as_in_context(ctx)
    segment_ids = segment_ids.as_in_context(ctx)
    label = label.as_in_context(ctx)
    
     # Forward computation
    pred = bert_classifier(token_ids, segment_ids, valid_length.astype('float32'))
    probabilities.append(pred)
    metric.update([label], [pred])

    #print(metric)

## Output Transformation
Perform the same output transformation as in the BERT training.

In [25]:
import numpy as np 
import pandas as pd
numpy_prob = [probabilities[i].asnumpy() for i in range(len(probabilities))] 

#convert the numpy array into a list
predictions = [numpy_prob[i].tolist() for i in range(len(numpy_prob))] 

In [26]:
# convert predictions as a dataframe and pickle the file
scibert_output = pd.DataFrame(predictions)
scibert_output.to_pickle('scibert_predictions.pkl')